<a href="https://colab.research.google.com/github/pabrey/datasciencecoursera/blob/master/4a_all_routes_mean_input_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previous

In [0]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
import json
from google.colab import files
import time
from datetime import datetime
import math

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create auxilary functions

## Create auxilary functions

### confgiuration file

In [0]:
import json

def create_single_conf_deterministic_node(mode, id, type_node, resolution, prority, resources,  releasingNodes = None ):
  node =  {
      "Mode": mode,
      "Id": id,
      "Type": type_node,
      "Distribution": [
        {
          "DistributionType": {
            "SamplesEstimation": {
              "Resolution": resolution
            }
          },
          "Priority": prority
        }
      ]
    }
  if releasingNodes!= None:
     node.update({"ReleasingNodes": [releasingNodes]})
  node.update({"Resources": resources})
  return node


def create_conf_list_nodes(node_list, resolution, prority, resources):
  list_node = []
  #to avoid duplicte create dictionary
  node_dic = {}
  for node in node_list:
    if (node[1] not in node_dic):
      node_dic[node[1]]= node
  
  for (key, node) in node_dic.items():
    #node variables: mode, id, type, resloution, priority
    list_node.append(create_single_conf_deterministic_node(node[0], node[1], node[2],  resolution, prority, resources))
  return list_node
  

def create_config_file(list_nodes, fin_node):
  x = {
    "ResultTimeDistance": 3600,
    "ArrivalEstimate": 
      {
        "ModelRelativePath": "src/main/resources/R/ratesEstimator/model.net",
        "ScaleFactor": 21
      }, 
    "Nodes": list_nodes,
    "SinkId": fin_node
  }
  # convert into JSON:
  return json.dumps(x)

resources = 1
resolution = 1000
# the result is a JSON string:
#print(create_base_file())
#print(create_single_deterministic_node("arrival", "LlegadaPersonas" ,1, 20, 0, 1))
list_node = []
list_node.append(("arrival", "LlegadaPersonas" ,1, 10000, 0, 1))
list_node.append(("service", "Tornos" ,5, 10000, 0, 2))
list_node.append(("service", "Tornos" ,5, 10000, 0, 2))
list_node.append(("service", "PassiveNode" ,2, 10000, 0, 1, "Tornos"))
list_node.append(("service", "ActividadPersona" ,10000, 5, 0, 1))
list_node.append(("service", "FinPersona" ,10000, 5, 0, 1))
resolution_test = 1000
prority_test = 1
resources_test = 1

last_node = list_node.pop()
list_send = create_conf_list_nodes(list_node, resolution_test, prority_test, resources_test)
result = create_config_file(list_send, last_node[1])

print(result)

{"ResultTimeDistance": 3600, "ArrivalEstimate": {"ModelRelativePath": "src/main/resources/R/ratesEstimator/model.net", "ScaleFactor": 21}, "Nodes": [{"Mode": "arrival", "Id": "LlegadaPersonas", "Type": 1, "Distribution": [{"DistributionType": {"SamplesEstimation": {"Resolution": 1000}}, "Priority": 1}], "Resources": 1}, {"Mode": "service", "Id": "Tornos", "Type": 5, "Distribution": [{"DistributionType": {"SamplesEstimation": {"Resolution": 1000}}, "Priority": 1}], "Resources": 1}, {"Mode": "service", "Id": "PassiveNode", "Type": 2, "Distribution": [{"DistributionType": {"SamplesEstimation": {"Resolution": 1000}}, "Priority": 1}], "Resources": 1}, {"Mode": "service", "Id": "ActividadPersona", "Type": 10000, "Distribution": [{"DistributionType": {"SamplesEstimation": {"Resolution": 1000}}, "Priority": 1}], "Resources": 1}], "SinkId": "FinPersona"}


## routes file

In [0]:
def create_file_routes(tinit, tsimulation, tmaxsimulation, samples):
  x = {
    "InitialData": {
      "SimulationConfig": {
        "t_init": tinit,
        "t_simulation": tsimulation,
        "t_max_simulation": tmaxsimulation
      },
      "Samples": {
          "User": samples
      },  
      "ArrivalSamples": {
        "ArrivalTime": []
      },
      "Resource_update": {}
    }
  }
  return json.dumps(x)



def create_routes_list_nodes(input_nodes):
  
  def add_data(nodes_user):  
    
      #ouput
      tem_sal = []
     
      
      lista_nodes_routes = nodes_user.values.tolist()
     # last_node = lista_nodes_routes.pop()
      for  node in lista_nodes_routes:       # print(node) 15
        tem_sal.append({ "NodeId": node[0], "ArrivalTime": node[1], "ServiceStartTime": node[2], "DepartureTime": node[3] })  
        
      #tem_sal.append({ "NodeId":last_node[0]})
      #factor of replication

      replication = int(lista_nodes_routes[0][4])
      
      tem_user =    [{"Priority": 1,"Step": tem_sal }]  
      return tem_user*replication
      
  sal_nodes = input_nodes.groupby("CASE_ID")[ "ACTIVITY", "ArrivalTime", "ServiceStartTime", "DepartureTime", "factor"].apply(lambda x: add_data(x))
  
  tem_sal = []
  for user_node in sal_nodes:
    tem_sal.extend(user_node)
  
  return tem_sal


data = [
    ["1", "LlegadaPersonas", 100, 100, 120, 2],
    ["1", "Tornos", 100, 100, 120, 2],
    ["1", "PassiveNode", 100, 100, 120, 2],
    ["1", "ActividadPersona", 100, 100, 120, 2], 
    ["1", "Tornos", 100, 100, 120, 2],
    ["1", "Fin"],
]
input_nodes = pd.DataFrame(data, columns = ["CASE_ID", "ACTIVITY", "ArrivalTime", "ServiceStartTime", "DepartureTime", "factor"])
list_nodes = create_routes_list_nodes(input_nodes  )
list_nodes
#result = create_file_routes(1, 60, 60, list_nodes)
#print(result)

[{'Priority': 1,
  'Step': [{'ArrivalTime': 100.0,
    'DepartureTime': 120.0,
    'NodeId': 'LlegadaPersonas',
    'ServiceStartTime': 100.0},
   {'ArrivalTime': 100.0,
    'DepartureTime': 120.0,
    'NodeId': 'Tornos',
    'ServiceStartTime': 100.0},
   {'ArrivalTime': 100.0,
    'DepartureTime': 120.0,
    'NodeId': 'PassiveNode',
    'ServiceStartTime': 100.0},
   {'ArrivalTime': 100.0,
    'DepartureTime': 120.0,
    'NodeId': 'ActividadPersona',
    'ServiceStartTime': 100.0},
   {'ArrivalTime': 100.0,
    'DepartureTime': 120.0,
    'NodeId': 'Tornos',
    'ServiceStartTime': 100.0},
   {'ArrivalTime': nan,
    'DepartureTime': nan,
    'NodeId': 'Fin',
    'ServiceStartTime': nan}]},
 {'Priority': 1,
  'Step': [{'ArrivalTime': 100.0,
    'DepartureTime': 120.0,
    'NodeId': 'LlegadaPersonas',
    'ServiceStartTime': 100.0},
   {'ArrivalTime': 100.0,
    'DepartureTime': 120.0,
    'NodeId': 'Tornos',
    'ServiceStartTime': 100.0},
   {'ArrivalTime': 100.0,
    'DepartureTime

## file join data functions

In [0]:
def create_final_files(input_dataframe, t_init, t_simulation, t_max_simulation, resolution, prority, resources):
  df_nodes = input_dataframe[["CASE_ID", "ACTIVITY", "ArrivalTime", "ServiceStartTime", "DepartureTime", "factor"]].drop_duplicates()
  list_nodes_routes = create_routes_list_nodes(df_nodes )
  routes = create_file_routes(t_init, t_simulation, t_max_simulation, list_nodes_routes)
  
  filter_nodes = input_dataframe[["Mode", "ACTIVITY", "Type", "Resolution", "Priority", "Resources", "INDEX" ]].drop_duplicates().sort_values(["INDEX"])
  lista_nodes_cfg = filter_nodes.values.tolist()
  last_node = lista_nodes_cfg.pop()
  
  
  create_conf_list_nodes(list_node, resolution_test, prority_test, resources_test)
  
  list_send = create_conf_list_nodes(lista_nodes_cfg,  resolution, prority, resources)
  cfg = create_config_file(list_send, last_node[1])
  
  return(routes, cfg)

# Create example with data from prosae

### load data

In [0]:
Location = 'drive/My Drive/data/prosae/original-log.csv'
Separator = ';'
df =  pd.read_csv(Location, sep=Separator)
df.columns = ["CASE_ID", "ACTIVITY",	"DATE",	"INDEX",	"COLOR"]
df_order = df.sort_values(['CASE_ID', 'INDEX'])
df_order.head()

,CASE_ID,ACTIVITY,DATE,INDEX,COLOR
0,173000001,ADMISION,01/01/2017 0:06,0,NaN
3,173000001,TRIAJE,01/01/2017 0:11,1,amarillo
16,173000001,ALTA,01/01/2017 0:58,2,NaN
1,173000002,ADMISION,01/01/2017 0:07,0,NaN
4,173000002,TRIAJE,01/01/2017 0:13,1,verde


In [0]:
df_order.head(1)

,CASE_ID,ACTIVITY,DATE,INDEX,COLOR
0,173000001,ADMISION,01/01/2017 0:06,0,NaN


In [0]:
df_order.tail(1)

,CASE_ID,ACTIVITY,DATE,INDEX,COLOR
511875,173181031,ALTA,01/01/2018 2:35,2,NaN


### create routes frame

In [0]:

# creamos  las columnas
# join function => The join() method takes all items in an iterable and joins them into one string.
# reset _index -> Pandas reset_index() is a method to reset index of a Data Frame. reset_index() method sets a list of integer ranging from 0 to length of data as index.
df_routes = df_order.groupby('CASE_ID')['ACTIVITY'].agg(lambda x: ','.join(x)).reset_index()
df_routes['SIZE'] = df_routes['ACTIVITY'].apply(lambda x: len(x.split(',')))

df_routes.columns = ['CASE_ID', 'PATH', 'SIZE_PATH']
df_routes['USER_PER_SIZE_PATH'] = df_routes.groupby('SIZE_PATH')["CASE_ID"].transform(len)
df_routes['USER_PER_PATH'] = df_routes.groupby('PATH')["CASE_ID"].transform(len)
df_routes.head()
df_routes.reset_index()
df_routes.head()

,CASE_ID,PATH,SIZE_PATH,USER_PER_SIZE_PATH,USER_PER_PATH
0,173000001,"ADMISION,TRIAJE,ALTA",3,50387,50387
1,173000002,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,47530
2,173000003,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,47530
3,173000005,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,47530
4,173000007,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,47530


let's check routes size

In [0]:
print("min size path: "+str(min(df_routes["SIZE_PATH"])))
print("Max size path: "+str(max(df_routes["SIZE_PATH"])))
print("Mean size path: "+str(df_routes["SIZE_PATH"].mean()))
print("MEdian size path: "+str(df_routes["SIZE_PATH"].median()))
print("Num users per route mean: "+str(df_routes["USER_PER_SIZE_PATH"].mean()))
print("Num users per route median: "+str(df_routes["USER_PER_SIZE_PATH"].median()))
number_user = df_routes["USER_PER_SIZE_PATH"].drop_duplicates().sum()
print("Num users : " + str(number_user))
df_routes["PERCENTAGE_PER_SIZE"] = df_routes["USER_PER_SIZE_PATH"]/number_user
df_routes["PERCENTAGE_PER_PATH"] = df_routes["USER_PER_PATH"]/number_user
df_routes[["SIZE_PATH", "USER_PER_SIZE_PATH", "PERCENTAGE_PER_SIZE"]].sort_values("SIZE_PATH").drop_duplicates()

min size path: 3
Max size path: 17
Mean size path: 4.42453417222664
MEdian size path: 5.0
Num users per route mean: 42443.59784975974
Num users per route median: 47810.0
Num users : 115708


,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE
0,3,50387,0.435467
37082,4,1011,0.008738
77339,5,47810,0.413195
109401,6,7557,0.065311
2114,7,2746,0.023732
36664,8,4450,0.038459
97922,9,761,0.006577
34457,10,537,0.004641
51915,11,255,0.002204
104999,12,89,0.000769


In [0]:
df_routes[["PATH","SIZE_PATH",  "USER_PER_SIZE_PATH", "PERCENTAGE_PER_SIZE", "USER_PER_PATH", "PERCENTAGE_PER_PATH"]].drop_duplicates().sort_values("PERCENTAGE_PER_PATH", ascending = False).head(15)

,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH
0,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467
1,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775
9,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,RX...",6,7557,0.065311,7001,0.060506
130,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,RX...",8,4450,0.038459,3314,0.028641
17,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,RX...",7,2746,0.023732,1426,0.012324
24,"ADMISION,TRIAJE,RX SOLICITUD,ALTA",4,1011,0.008738,841,0.007268
51,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,RX...",8,4450,0.038459,581,0.005021
88,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,IN...",7,2746,0.023732,581,0.005021
409,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,RX...",7,2746,0.023732,498,0.004304
31,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,RX...",6,7557,0.065311,344,0.002973


In [0]:
#check safety, sum must be 1
print(sum(df_routes.groupby("SIZE_PATH")["PERCENTAGE_PER_SIZE"].min()))
print(sum(df_routes.groupby("PATH")["PERCENTAGE_PER_PATH"].min()))

1.0
1.00000000000001


### and join data to create date and routes together

and join data beacuase we want also the size and number of elements

In [0]:
df_join = df.merge(df_routes).sort_values(["CASE_ID", "DATE"])
df_join =  df_join[["CASE_ID", "ACTIVITY", "DATE", "INDEX", "PATH", "SIZE_PATH", "USER_PER_SIZE_PATH", "PERCENTAGE_PER_SIZE",  "USER_PER_PATH", "PERCENTAGE_PER_PATH"]]
df_join = df_join.sort_values(['CASE_ID', 'INDEX'])
df_join.head()

,CASE_ID,ACTIVITY,DATE,INDEX,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH
0,173000001,ADMISION,01/01/2017 0:06,0,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467
1,173000001,TRIAJE,01/01/2017 0:11,1,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467
2,173000001,ALTA,01/01/2017 0:58,2,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467
3,173000002,ADMISION,01/01/2017 0:07,0,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775
4,173000002,TRIAJE,01/01/2017 0:13,1,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775


## process times

add timestamp and data

In [0]:
df_timestamp = df_join.copy()
#filter just the first id
#df_timestamp['TIMESTAMP'] = (pd.DatetimeIndex(df_timestamp.DATE).asi8)/10000000000
df_timestamp["TIMESTAMP"] = (df_timestamp['DATE']).apply(lambda x:  time.mktime( datetime.strptime(str(x), '%d/%m/%Y %H:%M').timetuple()))

df_timestamp["Mode"] =  np.where(df_timestamp["INDEX"] == 0, "arrival", "service")
df_timestamp["Type"] =  np.where(df_timestamp["INDEX"] == 0, 1, 2)
df_timestamp["Resolution"] = 1000
df_timestamp["Priority"] = 0
df_timestamp["Resources"] = 1
df_timestamp["ArrivalTime"] = df_timestamp["TIMESTAMP"] 
df_timestamp["ServiceStartTime"] =  df_timestamp["TIMESTAMP"] 
df_timestamp["DepartureTime"] = 0

df_timestamp.head()


,CASE_ID,ACTIVITY,DATE,INDEX,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH,TIMESTAMP,Mode,Type,Resolution,Priority,Resources,ArrivalTime,ServiceStartTime,DepartureTime
0,173000001,ADMISION,01/01/2017 0:06,0,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483229e+09,arrival,1,1000,0,1,1.483229e+09,1.483229e+09,0
1,173000001,TRIAJE,01/01/2017 0:11,1,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483229e+09,service,2,1000,0,1,1.483229e+09,1.483229e+09,0
2,173000001,ALTA,01/01/2017 0:58,2,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483232e+09,service,2,1000,0,1,1.483232e+09,1.483232e+09,0
3,173000002,ADMISION,01/01/2017 0:07,0,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775,1.483229e+09,arrival,1,1000,0,1,1.483229e+09,1.483229e+09,0
4,173000002,TRIAJE,01/01/2017 0:13,1,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775,1.483230e+09,service,2,1000,0,1,1.483230e+09,1.483230e+09,0


normalize times

In [0]:
min_value =  df_timestamp["ArrivalTime"].min()-1
df_timestamp["ArrivalTime"] = df_timestamp["ArrivalTime"]-min_value
df_timestamp["ServiceStartTime"] = df_timestamp["ServiceStartTime"]-min_value
df_timestamp.head()

,CASE_ID,ACTIVITY,DATE,INDEX,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH,TIMESTAMP,Mode,Type,Resolution,Priority,Resources,ArrivalTime,ServiceStartTime,DepartureTime
0,173000001,ADMISION,01/01/2017 0:06,0,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483229e+09,arrival,1,1000,0,1,1.0,1.0,0
1,173000001,TRIAJE,01/01/2017 0:11,1,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483229e+09,service,2,1000,0,1,301.0,301.0,0
2,173000001,ALTA,01/01/2017 0:58,2,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483232e+09,service,2,1000,0,1,3121.0,3121.0,0
3,173000002,ADMISION,01/01/2017 0:07,0,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775,1.483229e+09,arrival,1,1000,0,1,61.0,61.0,0
4,173000002,TRIAJE,01/01/2017 0:13,1,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775,1.483230e+09,service,2,1000,0,1,421.0,421.0,0


establish departure time

In [0]:
data_time = df_timestamp.sort_values(["CASE_ID", "INDEX"]).reset_index().copy()


def processDepartureTime(nodes):

  temp_time = 0
  len_nodes = len(nodes)
  sal = []
  mi_index = 0
  for index, node in nodes.iteritems():
    mi_index = mi_index +1
  
    if (mi_index == len_nodes):
      sal.append(node)
    else:
      sal.append((nodes[index+1]))

  return sal
      

data_time["DepartureTime"] = data_time.groupby("CASE_ID")["ServiceStartTime"].transform(lambda x: processDepartureTime(x))
data_time['Difference'] = data_time['DepartureTime'] - data_time['ServiceStartTime']
data_time.head()


#data_time["DepartureTime"].fillna(0, inplace=True)


,index,CASE_ID,ACTIVITY,DATE,INDEX,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH,TIMESTAMP,Mode,Type,Resolution,Priority,Resources,ArrivalTime,ServiceStartTime,DepartureTime,Difference
0,0,173000001,ADMISION,01/01/2017 0:06,0,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483229e+09,arrival,1,1000,0,1,1.0,1.0,301.0,300.0
1,1,173000001,TRIAJE,01/01/2017 0:11,1,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483229e+09,service,2,1000,0,1,301.0,301.0,3121.0,2820.0
2,2,173000001,ALTA,01/01/2017 0:58,2,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483232e+09,service,2,1000,0,1,3121.0,3121.0,3121.0,0.0
3,3,173000002,ADMISION,01/01/2017 0:07,0,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775,1.483229e+09,arrival,1,1000,0,1,61.0,61.0,421.0,360.0
4,4,173000002,TRIAJE,01/01/2017 0:13,1,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775,1.483230e+09,service,2,1000,0,1,421.0,421.0,1081.0,660.0


In [0]:
data_time["USER_PER_SIZE_PATH"].unique()

array([50387, 47810,   761,  7557,  4450,  2746,  1011,   537,    54,
          32,   255,    89,    13,     4,     2])

safety check, check the data, al difference is > 0

In [0]:
id_neg = data_time[data_time["Difference"] < 0]["CASE_ID"]
len(id_neg)

0

## search for inconstancies

alta difference always must be 0

In [0]:
df_filter = data_time.copy().sort_values(["CASE_ID", "INDEX"])
df_filter[(df_filter["ACTIVITY"] == "ALTA") & (df_filter["Difference"]>0)]

,index,CASE_ID,ACTIVITY,DATE,INDEX,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH,TIMESTAMP,Mode,Type,Resolution,Priority,Resources,ArrivalTime,ServiceStartTime,DepartureTime,Difference


last element must be ALTA

In [0]:
df_filter.tail(1)

,index,CASE_ID,ACTIVITY,DATE,INDEX,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH,TIMESTAMP,Mode,Type,Resolution,Priority,Resources,ArrivalTime,ServiceStartTime,DepartureTime,Difference
511953,511953,173181031,ALTA,01/01/2018 2:35,2,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.514774e+09,service,2,1000,0,1,31544941.0,31544941.0,31544941.0,0.0


all CASE_ID must have a ADMISION and a ALTA

In [0]:
df_general = df_filter["CASE_ID"].unique()
df_ALTA = df_filter[df_filter["ACTIVITY"] == "ALTA"]["CASE_ID"].unique()
set(df_general) - set(df_ALTA)

set()

In [0]:
df_general = df_filter["CASE_ID"].unique()
df_ALTA = df_filter[df_filter["ACTIVITY"] == "ADMISION"]["CASE_ID"].unique()
set(df_general) - set(df_ALTA)

set()

just for the decision of filter get the len vector

In [0]:
df_filter["USER_PER_SIZE_PATH"].unique()

array([50387, 47810,   761,  7557,  4450,  2746,  1011,   537,    54,
          32,   255,    89,    13,     4,     2])

# Create simulation auxiliary functions

## defini functions

In [0]:

#create data for the files
def launch_create_simulation_file(df_input, per_path_min, per_size_min):
  t_init =  161476390
  t_simulation =  600000
  t_max_simulation = 600000
  resources = 200
  resolution = 1000
  priority = 0


  #filter just with a minimul leng
  print("Number of inputs before filter: "+str(len(df_input)))
  data_print = df_input[df_input["PERCENTAGE_PER_PATH"]> percentage_path_min]
  data_print = data_print[data_print["PERCENTAGE_PER_SIZE"]> percentage_size_min]
  print("Number of inputs after filter: "+str(len(data_print)))

  #we just want an example for each path, so we create just one with the mean values
  data_print = data_print.groupby(["PATH", "ACTIVITY"]).mean().reset_index()
  #column mode is lost, maybe because is categorical
  data_print["Mode"] =  np.where(data_print["INDEX"] == 0, "arrival", "service")

  #and now we add the normalizer factor, each path gets a weigth depending of the len
  max_len = max(df_filter["USER_PER_SIZE_PATH"].unique())  
  data_print["factor"]= ((data_print["USER_PER_SIZE_PATH"]/max_len)*10).apply(np.ceil) 

  data_print = data_print.sort_values(["SIZE_PATH", "CASE_ID", "INDEX"])
  return data_print


#create data for stadistics
def create_data_average(input_df):
  def average_time(nodes):
      min_date = None
      max_date = None

      for index, node in nodes.iterrows():      
          temp_date = node["ServiceStartTime"] 
          if ((min_date == None) or min_date>temp_date):
            min_date = temp_date
          if ((max_date == None) or max_date<temp_date):
            max_date = temp_date

      return max_date- min_date
  out_data = input_df.groupby("CASE_ID")["CASE_ID", "ServiceStartTime", "Difference"].apply(lambda x : average_time(x))
  return out_data


#show estadistics
def print_stadistic_df(input_df):
  data_average = create_data_average(input_df)
  print("Average")
  print(data_average.head(10))
  print("Mean value of all the avergaes: "+str(data_average.mean()))
  print("Mean of the difference field:"+ str(input_df["Difference"].mean()))
  print("Mean of the difference in the nodes")
  print(input_df.groupby("ACTIVITY").mean()["Difference"])

## sinulation_00
percentage_path_min = 0
percentage_size_min = 0

In [0]:
percentage_path_min = 0
percentage_size_min = 0
data_print = launch_create_simulation_file(df_filter, percentage_path_min, percentage_size_min )


Number of inputs before filter: 511954
Number of inputs after filter: 511954


In [0]:
#create condifgurration files
(result_routes, result_cfg) = create_final_files(data_print, t_init, t_simulation, t_max_simulation, resolution, priority, resources)
print(result_routes)
print(result_cfg)

{"InitialData": {"SimulationConfig": {"t_init": 161476390, "t_simulation": 600000, "t_max_simulation": 600000}, "Samples": {"User": [{"Priority": 1, "Step": [{"NodeId": "ADMISION", "ArrivalTime": 105361.0, "ServiceStartTime": 105361.0, "DepartureTime": 105601.0}, {"NodeId": "TRIAJE", "ArrivalTime": 105601.0, "ServiceStartTime": 105601.0, "DepartureTime": 111301.0}, {"NodeId": "RX SOLICITUD", "ArrivalTime": 123761.0, "ServiceStartTime": 123761.0, "DepartureTime": 137641.0}, {"NodeId": "RX EXPLORACION", "ArrivalTime": 125621.0, "ServiceStartTime": 125621.0, "DepartureTime": 127121.0}, {"NodeId": "INTERCONSULTA", "ArrivalTime": 146611.0, "ServiceStartTime": 146611.0, "DepartureTime": 162421.0}, {"NodeId": "RX INFORME", "ArrivalTime": 126361.0, "ServiceStartTime": 126361.0, "DepartureTime": 139981.0}, {"NodeId": "RESPUESTA", "ArrivalTime": 162931.0, "ServiceStartTime": 162931.0, "DepartureTime": 180931.0}, {"NodeId": "ALTA", "ArrivalTime": 238681.0, "ServiceStartTime": 238681.0, "Departure

show results

In [0]:
 
print_stadistic_df(data_print)

Average
CASE_ID
173000537.0    133320.0
173000664.0    182760.0
173001251.0    196500.0
173001252.0    136140.0
173001423.0     73440.0
173001735.0     40980.0
173002388.0     70440.0
173002514.0     82860.0
173003720.0    182820.0
173004020.0    149460.0
dtype: float64
Mean value of all the avergaes: 97816.8978057838
Mean of the difference field:8395.120451339344
Mean of the difference in the nodes
ACTIVITY
ADMISION            731.034929
ALTA                  0.000000
INTERCONSULTA     15454.174855
RESPUESTA         15192.525683
RX EXPLORACION    14745.835534
RX INFORME        17041.738599
RX SOLICITUD       5351.812467
TRIAJE             6170.004933
Name: Difference, dtype: float64


Resuldatos en java

System mean time: 1366079.1817434644

Nodes information

admision : 751.2999621008252

alta : NaN

interconsulta : 17149.385429731417

respuesta : 18381.68481728972

rx exploracion : 2872877.790212897

rx informe : 20739.763746284523

rx solicitud : 6752.787258603854

triaje : 8140.181706914992



Resultados en java para 6000000
System mean time: 2812592.4855426624

Nodes information

admision : 747.6296508757738

alta : NaN

interconsulta : 127743.2180718071

respuesta : 38876.30844513437

rx exploracion : 3103545.5167867243

rx informe : 23135.821598840495

rx solicitud : 32551.43550627584

triaje : 2808823.3745929995

## simulation_01
percentage_path_min = 0.1 percentage_size_min = 0.1

In [0]:
percentage_path_min = 0.1
percentage_size_min = 0.1
data_print = launch_create_simulation_file(df_filter, percentage_path_min, percentage_size_min )


Number of inputs before filter: 511954
Number of inputs after filter: 388811


In [0]:
(result_routes, result_cfg) = create_final_files(data_print, t_init, t_simulation, t_max_simulation, resolution, priority, resources)
print(result_routes)
print(result_cfg)

{"InitialData": {"SimulationConfig": {"t_init": 161476390, "t_simulation": 600000, "t_max_simulation": 600000}, "Samples": {"User": [{"Priority": 1, "Step": [{"NodeId": "ADMISION", "ArrivalTime": 15638288.739532927, "ServiceStartTime": 15638288.739532927, "DepartureTime": 15639064.870397644}, {"NodeId": "TRIAJE", "ArrivalTime": 15639064.870397644, "ServiceStartTime": 15639064.870397644, "DepartureTime": 15643021.853355775}, {"NodeId": "RX SOLICITUD", "ArrivalTime": 15643021.853355775, "ServiceStartTime": 15643021.853355775, "DepartureTime": 15646156.044813802}, {"NodeId": "RX EXPLORACION", "ArrivalTime": 15646156.044813802, "ServiceStartTime": 15646156.044813802, "DepartureTime": 15667871.37660425}, {"NodeId": "ALTA", "ArrivalTime": 15667871.37660425, "ServiceStartTime": 15667871.37660425, "DepartureTime": 15667871.37660425}]}, {"Priority": 1, "Step": [{"NodeId": "ADMISION", "ArrivalTime": 15638288.739532927, "ServiceStartTime": 15638288.739532927, "DepartureTime": 15639064.870397644},

In [0]:
 print_stadistic_df(data_print)

Average
CASE_ID
1.730890e+08    29582.637071
1.730907e+08    18420.621589
dtype: float64
Mean value of all the avergaes: 24001.6293301126
Mean of the difference field:6000.40733252816
Mean of the difference in the nodes
ACTIVITY
ADMISION            771.745350
ALTA                  0.000000
RX EXPLORACION    21715.331790
RX SOLICITUD       3134.191458
TRIAJE            10805.122356
Name: Difference, dtype: float64


System mean time: 23530.00499502096

Nodes information

admision : 772.0696063153301

alta : NaN

rx exploracion : 21715.0

rx solicitud : 3134.0

triaje : 11856.271107059732

## simulation_0.05
percentage_path_min = 0.05 percentage_size_min = 0.05

In [0]:
percentage_path_min = 0.05
percentage_size_min = 0.05
data_print = launch_create_simulation_file(df_filter, percentage_path_min, percentage_size_min )

Number of inputs before filter: 511954
Number of inputs after filter: 430817


In [0]:
(result_routes, result_cfg) = create_final_files(data_print, t_init, t_simulation, t_max_simulation, resolution, priority, resources)
print(result_routes)
print(result_cfg)

{"InitialData": {"SimulationConfig": {"t_init": 161476390, "t_simulation": 600000, "t_max_simulation": 600000}, "Samples": {"User": [{"Priority": 1, "Step": [{"NodeId": "ADMISION", "ArrivalTime": 15638288.739532927, "ServiceStartTime": 15638288.739532927, "DepartureTime": 15639064.870397644}, {"NodeId": "TRIAJE", "ArrivalTime": 15639064.870397644, "ServiceStartTime": 15639064.870397644, "DepartureTime": 15643021.853355775}, {"NodeId": "RX SOLICITUD", "ArrivalTime": 15643021.853355775, "ServiceStartTime": 15643021.853355775, "DepartureTime": 15646156.044813802}, {"NodeId": "RX EXPLORACION", "ArrivalTime": 15646156.044813802, "ServiceStartTime": 15646156.044813802, "DepartureTime": 15667871.37660425}, {"NodeId": "ALTA", "ArrivalTime": 15667871.37660425, "ServiceStartTime": 15667871.37660425, "DepartureTime": 15667871.37660425}]}, {"Priority": 1, "Step": [{"NodeId": "ADMISION", "ArrivalTime": 15638288.739532927, "ServiceStartTime": 15638288.739532927, "DepartureTime": 15639064.870397644},

In [0]:
 print_stadistic_df(data_print)

Average
CASE_ID
1.730890e+08    29582.637071
1.730907e+08    18420.621589
1.730909e+08    36478.705899
dtype: float64
Mean value of all the avergaes: 28160.654853127275
Mean of the difference field:6034.426039955896
Mean of the difference in the nodes
ACTIVITY
ADMISION            775.691015
ALTA                  0.000000
RX EXPLORACION    13447.350226
RX INFORME        19638.648764
RX SOLICITUD       3816.521525
TRIAJE             9329.499749
Name: Difference, dtype: float64


System mean time: 12024.348156289174

Nodes information
admision : 772.9048014061583

alta : NaN

rx exploracion : NaN

rx informe : NaN

rx solicitud : NaN

triaje : 11430.42983620879


# Auxiliar downlaod

In [0]:
open('result_routes.temp', 'w').write(result_routes)
files.download('result_routes.temp')

In [0]:
open('result_config.temp', 'w').write(result_cfg)
files.download('result_config.temp')